# CSCI 6920 - Homework 6
Name: Ohad Nir;
Due: 11/11/2022

## Task 1
### Prompt:
We discussed how we can formulate RL problems as an MDP. Describe any real-world application that can be formulated as an MDP. Describe the state space, action space, transition model, and rewards for that problem. You do not need to be precise in the description of the transition model and reward (no formula is needed). Qualitative description is enough.

### Response:
A possible real-world application of MDPs is in deer population control. The state of New York wants to issue a number of deer allowed to be hunted this season. They don't want there to be too many deer such that they eat all the vegetation, and they don't want there to be too few that they go extinct. The MDP problem can be framed where the state is the population of deer. Actions are the amount of deer allowed to be hunted. Rewards are the ecological disruption, i.e., a lower population and a high population have a lower score while the Mid-sized population has a high score. The transition model is where there is a higher probability for the population to go up if the amount of deer hunting is low, while there is a higher likelihood of the population to go down if deer hunting is high.

## Task 2
### Prompt:
RL is used in various sectors - Healthcare, recommender systems and trading are a few of those. Pick one of the three areas. Explain one of the problems in any of these domains that can be more effectively solved by reinforcement learning. Find an open-source project (if any) that has addressed this problem. Explain this project in detail.
### Response:
An aspect of trading that a reinforcement learning model can better solve is stock path prediction for option pricing using the Monte Carlo method. To price an option, you need to generate a lot of possible stock paths based on the price dynamics. Finding the values that make up the dynamics takes time and could be sped up by reinforcement learning through predicting the risk-neutral stock path very quickly, which can then be used in the Monte Carlo method. This repo implements this: https://github.com/bharatpurohit97/StockPrediction

## Task 3

Evaluation Metric: I used win rate as the evalutation metric for this assignment. This metric is optained by speisifing the number of games to play against a random player and computing the number of wins the AI gets in those games.

### Setup

In [14]:
import os
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

### Board and Agent Classes

In [340]:
class TicTacToeAgent():
    def __init__(self, seed=None):
        self.q_table = dict()
        
        if seed:
            np.random.seed(seed)
            
    def move(self, board):
        board_hash = self.hash_state(board.state())
        if board_hash in self.q_table:
            a = np.argmax(self.q_table[self.hash_state(board.state())])
        else:
            a = np.random.choice(board.available_moves())
            
        return (a // 3, a % 3)
    
    def train(self, episodes, alpha, gamma, epsilon):
        board = TicTacToeBoard()
        for e in tqdm(range(episodes)):
            s = self.add_state(board.state()) # inital state
            while(board.winner() == 0):
                a = self.action(s, board.available_moves(), epsilon) # get next action.
                board.move(a // 3, a % 3) # move 
                
                if len(board.available_moves()) != 0: # random player makes move.
                    a = np.random.choice(board.available_moves())
                    board.move(a // 3, a % 3)

                s_p = self.add_state(board.state()) # new board state is added to the q-table.
                R = board.winner() # get reward if there is a winner.
                
                # apply learning step.
                n_max = np.argmax(self.q_table[s_p]) 
                self.q_table[s][a] += alpha * (R + gamma * n_max - self.q_table[s][a])
                
                s = s_p # set new state.
                
            board.reset()
    
    def evaluate(self, runs):
        board = TicTacToeBoard()
        wins = 0
        for r in tqdm(range(runs)):
            while(board.winner() == 0):
                r, c = self.move(board)
                board.move(r, c) # move 
                
                if len(board.available_moves()) != 0: # random player makes move.
                    a = np.random.choice(board.available_moves())
                    board.move(a // 3, a % 3)
            
            wins += 1 if board.winner() == 1 else 0
            board.reset()
        return wins / runs
                        
    def hash_state(self, state):
        return str(state)
            
    def add_state(self, state):
        state_hash = self.hash_state(state)
        if state_hash not in self.q_table:
            self.q_table[state_hash] = np.where(state == 0, 0, -np.Inf)
        return state_hash            
            
    def action(self, s_current, s_future, epsilon):
        if np.random.random() < epsilon:
            return np.random.choice(s_future) # random action
        return s_future[np.argmax(self.q_table[s_current][s_future])]


class TicTacToeBoard():
    def __init__(self):
        self.board = np.zeros((3,3))
        self.player = 1
        
    def reset(self):
        self.board = np.zeros((3,3))
        self.player = 1
        
    def state(self):
        return self.board.flatten()
        
    def move(self, r, c):
        if r < 0 or r > 2:
            print(f"Row {r} is out of bounds.")
            return False
        if c < 0 or c > 2:
            print(f"Column {c} is out of bounds.")
            return False 
        if self.board[r][c] != 0:
            print(f"Spot at at row {r} and column {c} has already been used.")
            return False
        
        self.board[r][c] = self.player
        self.player *= -1
        return True
        
    def available_moves(self):
        flat_board = np.ravel(self.board)
        return np.where(flat_board == 0)[0]
    
    def winner(self):
        sum_col = np.sum(self.board, axis=0)
        sum_row = np.sum(self.board, axis=1)
        sum_diag = np.trace(self.board)
        sum_odiag = np.sum(np.fliplr(self.board).diagonal())
        
        if (sum_col==3).any() or (sum_row==3).any() or sum_diag==3 or sum_odiag==3:
            return 1
        elif (sum_col==-3).any() or (sum_row==-3).any() or sum_diag==-3 or sum_odiag==-3:
            return -1
        elif (self.board!=0).all():
            return 0.1
        else:
            return 0
        
    def __str__(self):
        board_str = "+---------+\n"
        for r in range(3):
            board_str += "|"
            for c in range(3):
                if self.board[r][c] == 1:
                    board_str += " O "
                elif self.board[r][c] == 0:
                    board_str += " . "
                elif self.board[r][c] == -1:
                    board_str += " X "
                else:
                    return "Error..."
            board_str += "|\n"
        board_str += "+---------+"
        return board_str    

### Functions

In [341]:
def play(agent):
    ttt = TicTacToeBoard()
    print(ttt)
    count = 0
    while(ttt.winner() == 0):
        if ttt.player == 1:
            print("AI Move:")
            r, c = agent.move(ttt)
        else:
            print("Player Move:")
            r = int(input("Row:"))
            c = int(input("Column:"))

        ttt.move(r,c)
        print(ttt, "\n")

    if ttt.winner() == 1:
        print("AI wins!")
    elif ttt.winner() == -1:
        print("Player wins!")
    else:
        print("Tie!")

### Build and Train Agent

In [346]:
agent = TicTacToeAgent(2)
episodes = 10000
alpha = 0.1
gamma = 0.9
epsilon = 0.3

In [347]:
agent.train(episodes, alpha, gamma, epsilon)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [348]:
win_rate = agent.evaluate(episodes)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [350]:
print(f"The Average Win Rate of the Agent against a Random Player is {win_rate*100:.2f}%")

The Average Win Rate of the Agent against a Random Player is 57.47%


### Play!

In [326]:
play(agent)

+---------+
| .  .  . |
| .  .  . |
| .  .  . |
+---------+
AI Move:
+---------+
| .  .  . |
| .  O  . |
| .  .  . |
+---------+ 

Player Move:


Row: 1
Column: 2


+---------+
| .  .  . |
| .  O  X |
| .  .  . |
+---------+ 

AI Move:
+---------+
| .  .  O |
| .  O  X |
| .  .  . |
+---------+ 

Player Move:


Row: 2
Column: 0


+---------+
| .  .  O |
| .  O  X |
| X  .  . |
+---------+ 

AI Move:
+---------+
| .  .  O |
| O  O  X |
| X  .  . |
+---------+ 

Player Move:


Row: 2
Column: 1


+---------+
| .  .  O |
| O  O  X |
| X  X  . |
+---------+ 

AI Move:
+---------+
| O  .  O |
| O  O  X |
| X  X  . |
+---------+ 

Player Move:


Row: 2
Column: 2


+---------+
| O  .  O |
| O  O  X |
| X  X  X |
+---------+ 

Player wins!


In [331]:
play(agent)

+---------+
| .  .  . |
| .  .  . |
| .  .  . |
+---------+
AI Move:
+---------+
| .  .  . |
| .  O  . |
| .  .  . |
+---------+ 

Player Move:


Row: 0
Column: 0


+---------+
| X  .  . |
| .  O  . |
| .  .  . |
+---------+ 

AI Move:
+---------+
| X  O  . |
| .  O  . |
| .  .  . |
+---------+ 

Player Move:


Row: 2
Column: 1


+---------+
| X  O  . |
| .  O  . |
| .  X  . |
+---------+ 

AI Move:
+---------+
| X  O  . |
| O  O  . |
| .  X  . |
+---------+ 

Player Move:


Row: 1
Column: 2


+---------+
| X  O  . |
| O  O  X |
| .  X  . |
+---------+ 

AI Move:
+---------+
| X  O  . |
| O  O  X |
| O  X  . |
+---------+ 

Player Move:


Row: 0
Column: 2


+---------+
| X  O  X |
| O  O  X |
| O  X  . |
+---------+ 

AI Move:
+---------+
| X  O  X |
| O  O  X |
| O  X  O |
+---------+ 

Tie!


In [334]:
play(agent)

+---------+
| .  .  . |
| .  .  . |
| .  .  . |
+---------+
AI Move:
+---------+
| .  .  . |
| .  O  . |
| .  .  . |
+---------+ 

Player Move:


Row: 2
Column: 1


+---------+
| .  .  . |
| .  O  . |
| .  X  . |
+---------+ 

AI Move:
+---------+
| O  .  . |
| .  O  . |
| .  X  . |
+---------+ 

Player Move:


Row: 0
Column: 1


+---------+
| O  X  . |
| .  O  . |
| .  X  . |
+---------+ 

AI Move:
+---------+
| O  X  . |
| O  O  . |
| .  X  . |
+---------+ 

Player Move:


Row: 0
Column: 2


+---------+
| O  X  X |
| O  O  . |
| .  X  . |
+---------+ 

AI Move:
+---------+
| O  X  X |
| O  O  O |
| .  X  . |
+---------+ 

AI wins!
